In [82]:
%pip install --upgrade pip
%pip install boto3 prettytable

111295.80s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.
111303.83s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


In [83]:
import boto3
import subprocess
import json
import os
import datetime
import time
from prettytable import PrettyTable

## Step 1: Create session and client

In [84]:
def set_aws_credentials(profile, region_name='us-east-1'):
    result = subprocess.run(f"aws-vault exec {profile} --json", shell=True, capture_output=True)
    credentials = json.loads(result.stdout)

    # Create a session with the retrieved credentials
    session = boto3.session.Session(
        aws_access_key_id=credentials['AccessKeyId'],
        aws_secret_access_key=credentials['SecretAccessKey'],
        aws_session_token=credentials['SessionToken'],
        region_name=region_name        
    )

    return session



## Get Service Scaling Parameters and Status

In [85]:
def getScalingDetails(profile, region_name, cluster_name, service_name):
    # Use the function with your profile to get a session
    aws_session = set_aws_credentials(profile, region_name=region_name)

    # Create ECS client using the session
    ecs_client = aws_session.client('ecs')
    app_scaling_client = aws_session.client('application-autoscaling')

    # Fetch the details of the specified service
    response = ecs_client.describe_services(
        cluster=cluster_name,
        services=[service_name]
    )

    service_info = response['services'][0]
    desired_tasks = service_info['desiredCount']
    
    # Resource ID format for ECS services: service/<clusterName>/<serviceName>
    resource_id = f"service/{cluster_name}/{service_name}"

    # Check for scalable targets
    scalable_targets = app_scaling_client.describe_scalable_targets(
        ServiceNamespace='ecs',
        ResourceIds=[resource_id],
        ScalableDimension='ecs:service:DesiredCount'
    )
    
    if scalable_targets['ScalableTargets']:
        # Extract min and max tasks from the first scaling policy
        target = scalable_targets['ScalableTargets'][0]
        min_tasks = target['MinCapacity']
        max_tasks = target['MaxCapacity']
        return min_tasks, max_tasks, desired_tasks
    else:
        # No scaling policy, return the current desired count
        return desired_tasks, desired_tasks, desired_tasks


In [86]:
def getLoadBalancingAlgorithm(profile, region_name, cluster_name, service_name):
    # Use the function with your profile to get a session
    aws_session = set_aws_credentials(profile, region_name=region_name)

    # Create ECS and ELBv2 clients using the session
    ecs_client = aws_session.client('ecs')
    elbv2_client = aws_session.client('elbv2')

    # Fetch the details of the specified service
    response = ecs_client.describe_services(
        cluster=cluster_name,
        services=[service_name]
    )

    service_info = response['services'][0]
    tg_arns = [lb['targetGroupArn'] for lb in service_info['loadBalancers']]
    target_groups = elbv2_client.describe_target_groups(TargetGroupArns=tg_arns)['TargetGroups']
    for target_group in target_groups:
        if target_group['TargetGroupName'] == 'projects-main-fg':
                # Fetch attributes of the target group
                attributes = elbv2_client.describe_target_group_attributes(
                    TargetGroupArn=target_group['TargetGroupArn']
                )
                # Find and return the load balancing algorithm attribute
                for attr in attributes['Attributes']:
                    if attr['Key'] == 'load_balancing.algorithm.type':
                        lb_alg = 'LOR' if attr['Value'] == 'least_outstanding_requests' else attr['Value']
                        return lb_alg
    
    return 'Not Found'

In [87]:
def get_task_metrics(profile, region_name, cluster_name, service_name, delta_hours=12):
    aws_session = set_aws_credentials(profile, region_name=region_name)
    cloudwatch_client = aws_session.client('cloudwatch')
    
    # Get CPU utilization from CloudWatch
    cpu_utilization = cloudwatch_client.get_metric_statistics(
        Namespace='AWS/ECS',
        MetricName='CPUUtilization',
        Dimensions=[
            {'Name': 'ClusterName', 'Value': cluster_name},
            {'Name': 'ServiceName', 'Value': service_name}
        ],
        StartTime=datetime.datetime.utcnow() - datetime.timedelta(hours=delta_hours),
        EndTime=datetime.datetime.utcnow(),
        Period=300, # 60, 300, 3600
        Statistics=['Average'],
    )

    mem_utilization = cloudwatch_client.get_metric_statistics(
        Namespace='AWS/ECS',
        MetricName='MemoryUtilization',
        Dimensions=[
            {'Name': 'ClusterName', 'Value': cluster_name},
            {'Name': 'ServiceName', 'Value': service_name}
        ],
        StartTime=datetime.datetime.utcnow() - datetime.timedelta(hours=12),
        EndTime=datetime.datetime.utcnow(),
        Period=300, # 60, 300, 3600
        Statistics=['Average'],
    )
    
    
    # parse the response to get average, min, max
    cpu_utilization = cpu_utilization['Datapoints']
    cpu_utilization = [point['Average'] for point in cpu_utilization]
    cpu_avg = sum(cpu_utilization) / len(cpu_utilization)
    cpu_min = min(cpu_utilization)
    cpu_max = max(cpu_utilization)

    mem_utilization = mem_utilization['Datapoints']
    mem_utilization = [point['Average'] for point in mem_utilization]
    mem_avg = sum(mem_utilization) / len(mem_utilization)
    mem_min = min(mem_utilization)
    mem_max = max(mem_utilization)

    return cpu_avg, cpu_min, cpu_max, mem_avg, mem_min, mem_max
    #print("CPU Utilization:", round(cpu_avg, 2), "%", "(min:", round(cpu_min, 2), "%, max:", round(cpu_max, 2), "%)")

    

In [88]:
# Specify your cluster and service name
cluster_name = 'EcsCluster1-main'
service_name = 'projects-application-fargate-main'

environments = [
    ('acl-playground', 'us-west-2'),  # playground
    ('acl-staging', 'us-west-2'),  # staging
    ('acl-staging', 'us-east-1'),  # preprod?
    ('acl-production', 'us-east-1'),
    ('acl-production', 'eu-central-1'),
    ('acl-production', 'af-south-1'),
    ('acl-production', 'ca-central-1'),
    ('acl-production', 'ap-northeast-1'),
    ('acl-production', 'ap-southeast-1'),
    ('acl-production', 'ap-southeast-2'),
    ('acl-production', 'sa-east-1'),
]

table = PrettyTable()
table.field_names = ["Profile", "Region", "Min", "Max", "Act", "%", "Algorithm", "CPU Avg", "CPU Min", "CPU Max", "Mem Avg", "Mem Min", "Mem Max"]

for profile, region in environments:
    min_tasks, max_tasks, desired_tasks = getScalingDetails(profile, region, cluster_name, service_name)
    algorithm = getLoadBalancingAlgorithm(profile, region, cluster_name, service_name)
    percentage = str(round(desired_tasks/max_tasks*100.,2)) if max_tasks > min_tasks else "N/A"
    cpu_avg, cpu_min, cpu_max, mem_avg, mem_min, mem_max = get_task_metrics(profile, region, cluster_name, service_name)

    print(f"{profile} {region} - Actual Tasks: {desired_tasks} ({percentage}%) CPU: {round(cpu_avg, 2)}%  Memory: {round(mem_avg, 2)}%")
    table.add_row([profile, region, min_tasks, max_tasks, desired_tasks, percentage, algorithm, 
                   round(cpu_avg, 2), round(cpu_min, 2), round(cpu_max, 2), 
                   round(mem_avg, 2), round(mem_min, 2), round(mem_max, 2)])

print(table)

acl-playground us-west-2 - Actual Tasks: 1 (25.0%) CPU: 10.36%  Memory: 50.33%
acl-staging us-west-2 - Actual Tasks: 1 (25.0%) CPU: 16.25%  Memory: 65.84%
acl-staging us-east-1 - Actual Tasks: 3 (N/A%) CPU: 5.93%  Memory: 63.53%
acl-production us-east-1 - Actual Tasks: 33 (68.75%) CPU: 15.06%  Memory: 37.89%
acl-production eu-central-1 - Actual Tasks: 26 (52.0%) CPU: 10.91%  Memory: 45.18%
acl-production af-south-1 - Actual Tasks: 4 (12.5%) CPU: 11.04%  Memory: 65.53%
acl-production ca-central-1 - Actual Tasks: 4 (12.5%) CPU: 13.31%  Memory: 60.64%
acl-production ap-northeast-1 - Actual Tasks: 4 (12.5%) CPU: 12.3%  Memory: 53.0%
acl-production ap-southeast-1 - Actual Tasks: 6 (12.5%) CPU: 22.39%  Memory: 71.34%
acl-production ap-southeast-2 - Actual Tasks: 6 (12.5%) CPU: 2.63%  Memory: 31.41%
acl-production sa-east-1 - Actual Tasks: 6 (12.5%) CPU: 10.16%  Memory: 56.42%
+----------------+----------------+-----+-----+-----+-------+-----------+---------+---------+---------+---------+----